<a href="https://colab.research.google.com/github/VitalyGladyshev/gb_nlp/blob/main/HW_04_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ДЗ 04 NLP

В качестве заготовки для задания прогоним часть 2ого домашнего задания. Нам необходимо получить разреженные матрицы, используя CountVectorizer, TfidfVectorizer для 'tweet_stemmed' и 'tweet_lemmatized' столбцов (4 матрицы).

In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
path_nlp = "/content/gdrive/My Drive/Colab Notebooks/NLP/"

In [5]:
train_cor = pd.read_pickle(path_nlp + "train_cor.pkl")
test_cor = pd.read_pickle(path_nlp + "test_cor.pkl")

In [6]:
test_cor.tail(2)

,id,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
49157,49158,"happy, at work conference: right mindset leads...",happy at work conference right mindset leads t...,"[happy, at, work, conference, right, mindset, ...","[happy, work, conference, right, mindset, lead...","{mindset, happi, cultureofdevelop, organ, conf...","{mindset, happy, cultureofdevelopment, work, c..."
49158,49159,"my song ""so glad"" free download! #shoegaze ...",my song so glad free download shoegaze newmusi...,"[my, song, so, glad, free, download, shoegaze,...","[song, glad, free, download, shoegaze, newmusi...","{newmus, free, shoegaz, download, newsong, gla...","{free, download, newmusic, newsong, glad, shoe..."


In [7]:
test_cor['label'] = -1

In [8]:
data = train_cor.copy()
data = data.append(test_cor, ignore_index=True)
data.tail(2)

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
49157,49158,-1,"happy, at work conference: right mindset leads...",happy at work conference right mindset leads t...,"[happy, at, work, conference, right, mindset, ...","[happy, work, conference, right, mindset, lead...","{mindset, happi, cultureofdevelop, organ, conf...","{mindset, happy, cultureofdevelopment, work, c..."
49158,49159,-1,"my song ""so glad"" free download! #shoegaze ...",my song so glad free download shoegaze newmusi...,"[my, song, so, glad, free, download, shoegaze,...","[song, glad, free, download, shoegaze, newmusi...","{newmus, free, shoegaz, download, newsong, gla...","{free, download, newmusic, newsong, glad, shoe..."


In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
count_vectorizer_stemmed = CountVectorizer(stop_words='english', max_df=0.9, max_features=1000)
count_vectorizer_lemmatized = CountVectorizer(stop_words='english', max_df=0.9, max_features=1000)

bag_of_words_stemmed = count_vectorizer_stemmed.fit_transform(list(map(lambda x: " ".join(x), data['tweet_stemmed'].values)))
bag_of_words_lemmatized = count_vectorizer_lemmatized.fit_transform(list(map(lambda x: " ".join(x), data['tweet_lemmatized'].values)))

df_count_vect_stemmed = pd.DataFrame(bag_of_words_stemmed.toarray(), columns=count_vectorizer_stemmed.get_feature_names())
df_count_vect_lemmatized = pd.DataFrame(bag_of_words_lemmatized.toarray(), columns=count_vectorizer_lemmatized.get_feature_names())
df_count_vect_lemmatized.head(2)

,able,absolutely,accept,account,act,action,actor,actually,adapt,add,adventure,advice,affirmation,affirmations,afternoon,age,ago,agree,ahead,aist,album,alive,allahsoil,alligator,allow,altwaystoheal,amaze,america,american,americans,amp,anger,angry,animals,anniversary,announce,answer,anxiety,anymore,app,...,white,wife,wild,win,wine,winner,wish,woh,woman,women,wonder,wonderful,word,work,workout,world,worry,worse,worst,wow,write,wrong,wtf,xx,xxx,ya,yay,yeah,year,years,yes,yesterday,yo,yoga,york,young,youtube,yr,yrs,yummy
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
tfidf_vectorizer_stemmed = TfidfVectorizer(stop_words='english', max_df=0.9, max_features=1000)
tfidf_vectorizer_lemmatized = TfidfVectorizer(stop_words='english', max_df=0.9, max_features=1000)

bag_of_words_tfidf_stemmed = tfidf_vectorizer_stemmed.fit_transform(list(map(lambda x: " ".join(x), data['tweet_stemmed'].values)))
bag_of_words_tfidf_lemmatized = tfidf_vectorizer_lemmatized.fit_transform(list(map(lambda x: " ".join(x), data['tweet_lemmatized'].values)))

df_tfidf_vect_stemmed = pd.DataFrame(bag_of_words_tfidf_stemmed.toarray(), columns=tfidf_vectorizer_stemmed.get_feature_names())
df_tfidf_vect_lemmatized = pd.DataFrame(bag_of_words_tfidf_lemmatized.toarray(), columns=tfidf_vectorizer_lemmatized.get_feature_names())
df_tfidf_vect_lemmatized.head(2)

,able,absolutely,accept,account,act,action,actor,actually,adapt,add,adventure,advice,affirmation,affirmations,afternoon,age,ago,agree,ahead,aist,album,alive,allahsoil,alligator,allow,altwaystoheal,amaze,america,american,americans,amp,anger,angry,animals,anniversary,announce,answer,anxiety,anymore,app,...,white,wife,wild,win,wine,winner,wish,woh,woman,women,wonder,wonderful,word,work,workout,world,worry,worse,worst,wow,write,wrong,wtf,xx,xxx,ya,yay,yeah,year,years,yes,yesterday,yo,yoga,york,young,youtube,yr,yrs,yummy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Задание 1

Построим модель LogisticRegression, используя Bag-of-Words признаки для столбца combine_df['tweet_stemmed']. 
- Поделим Bag-of-Words признаки на train, test (train заканчивается на 31962 строке combine_df)
- Ответами является столбец train_df['label']
- Рассчитаем predict_proba, приведем prediction в в бинарный вид: если предсказание >= 0.3 то 1, иначе 0, тип заменим на int
- Рассчитаем f1_score 

Повторим аналогично для столбца combine_df['tweet_lemmatized'].

## Задание 2

Построим модель LogisticRegression, используя TF-IDF признаки для столбца combine_df['tweet_stemmed']. 
- Поделим TF-IDF признаки на train, test (train заканчивается на 31962 строке combine_df)
- Ответами является столбец train_df['label']
- Рассчитаем predict_proba, приведем prediction в в бинарный вид: если предсказание >= 0.3 то 1, иначе 0, тип заменим на int
- Рассчитаем f1_score 

Повторим аналогично для столбца combine_df['tweet_lemmatized'].

## Задание 3

Выведите результаты f1-score всех моделей, сделайте вывод.

## Задание 4

Теперь перейдем к визуализации. Посмотрим, какие слова являются наиболее популярные в датасете с помощью облака слов (WordCloud).
Облако слов - это визуализация, в которой наиболее частые слова большого размера, а менее частые слова меньшего размера.
- объединим слова в одну строку
- создадим словарь частот слов с помощью collections.Counter
- нарисуем облако слов с частотами слов с помощью WordCloud.generate_from_frequencies()
- используем nltk.corpus.stopwords как параметр stopwords, чтобы убрать "мусорные" частотные слова

## Задание 5

Теперь отобразим облако слов для отзывов, не содержащих токсичных комментариев (combine_df['label'] == 0).

## Задание 6

Теперь отобразим облако слов для отзывов, содержащих токсичные комментарии (combine_df['label'] == 1). 